<!-- TABS -->
# Perform a vector search

In [ ]:
from superduperdb import Document

item = Document({'x': datatype(sample_datapoint)})

Once we have this search target, we can execute a search as follows:

In [ ]:
# <tab: MongoDB>
select = collection.find().like(sample_datapoint)

In [ ]:
# <tab: SQL>
select = table.like(item)

In [ ]:
results = db.execute(select)